In [199]:
import os
import shutil
import random
import numpy as np
import statistics
import cv2
import json
import pickle
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon
from tqdm.notebook import tqdm
from operator import itemgetter

## 0. Helpers

****

In [320]:
# ----------------------------------------------------------------------
# function for reading the Json files
# ----------------------------------------------------------------------
def idx_finder(folder, orientation):
    '''
    This function takes the folder name (string),
    and an orientation adn returns a list of indices
    of the target orientation and valid frames in
    the folder.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder

    # read the json files
    # ------------------------------------------------------------------
    # read the Face Crop json
    with open(path + '/appleFace.json', 'r') as file:
        face = json.load(file)
    # read the Left Eye json
    with open(path + '/appleLeftEye.json', 'r') as file:
        eye_l = json.load(file)
    # read the Right Eye json
    with open(path + '/appleRightEye.json', 'r') as file:
        eye_r = json.load(file)
    # read the Scree json
    with open(path + '/screen.json', 'r') as file:
        screen = json.load(file)
    
    # make a list of all valid indices
    # ------------------------------------------------------------------
    # list of all frames with full face in it
    faces = [idx for idx, val in enumerate(face['IsValid']) if val == 1]
    # list of all frames with the left eye in it
    eyes_l = [idx for idx, val in enumerate(eye_l['IsValid']) if val == 1]
    # list of all frames with the right eye in it
    eyes_r = [idx for idx, val in enumerate(eye_r['IsValid']) if val == 1]
    # list of all frames with the desired orientation
    oriented = [idx for idx, ori in enumerate(screen['Orientation']) if ori == orientation]
    # intersection of all those lists
    indices = [idx for idx in faces if idx in eyes_l and idx in eyes_r and idx in oriented]
    
    return indices



# ----------------------------------------------------------------------
# a function for reading Json files
# ----------------------------------------------------------------------
def json_reader(dictionary, jsn, frame):
    '''
    The function takes a dictionary and a string mentioning
    the name of the json file, and a frame index, and it
    reads index-related info from json files and record them
    in the dictionary.
    '''
    # set the path to the json files
    path = 'Data_MIT/' + frame[:5] + '/'
    # get the index of the frame
    idx = int(frame[-5:])
    # read the face json file
    with open(str(path + 'apple' + jsn + '.json'), 'r') as file:
        jsn_info = json.load(file)    
    # add the face patch coordinates to the dictionary
    dictionary[jsn] = [round(jsn_info['X'][idx]),
                       round(jsn_info['Y'][idx]),
                       round(jsn_info['W'][idx]),
                       round(jsn_info['H'][idx])]


    
# ----------------------------------------------------------------------    
# a function to cut, resize, and save a patch
# ----------------------------------------------------------------------
def patch_reader(image, coordinates, path):
    '''
    The function takes an image and a list of coordinates (int),
    and a path (string), and cuts a patch out of the image,
    resizes it, and saves it in desired path.
    '''
    # check if the patches falls out of the frame
    # -------------------------------------------
    # whether the X coordinate is negative
    if coordinates[0] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, 0, 0, np.abs(coordinates[0]), 0, cv2.BORDER_REPLICATE)
        # and reset the X value regarding the new frame
        coordinates[0] = 0

    # whether Y is negative
    if coordinates[1] < 0:
        # if so, fill the shortage by replicating the edge 
        image = cv2.copyMakeBorder(image, np.abs(coordinates[1]), 0, 0, 0, cv2.BORDER_REPLICATE)
        # and reset the Y value regarding the new frame
        coordinates[1] = 0

    # whether X+W is larger than the frame width
    if coordinates[0]+coordinates[2] > image.shape[1]:
        # if so, fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[1]-(coordinates[0]+coordinates[2]))
        image = cv2.copyMakeBorder(image, 0, 0, 0, diff, cv2.BORDER_REPLICATE)

    # whether Y+H is larger than the frame height
    if coordinates[1]+coordinates[3] > image.shape[0]:
        # so we fill the shortage (diff) by replicating the edge
        diff = np.abs(image.shape[0]-(coordinates[1]+coordinates[3]))
        image = cv2.copyMakeBorder(image, 0, diff, 0, 0, 0, cv2.BORDER_REPLICATE)

        
    # cut the patch
    patch = image[coordinates[1]:coordinates[1]+coordinates[3],
                  coordinates[0]:coordinates[0]+coordinates[2]]
    
    # resize the patch
    patch_res = cv2.resize(patch, (64, 64))
    
    # save the image
    cv2.imwrite(path, patch_res);



# ----------------------------------------------------------------------
# a function for reading grid coordinates
# ----------------------------------------------------------------------
def get_coords(folder, frame):
    '''
    This function takes the folder and the frame names (string),
    read the faceGrid.json file, and returns the grid
    coordinates.
    '''
    # set the path to the grid json file
    path = 'Data_MIT/' + folder + '/faceGrid.json'
    # read the json file
    with open(path, 'r') as file:
        grid = json.load(file)
    # set the frame index
    idx = int(frame)
    # make a list of the X,Y,W, and H coordinates
    coords = [round(grid['X'][idx]), round(grid['Y'][idx]),
              round(grid['W'][idx]), round(grid['H'][idx])]
    
    # and return the coords list
    return coords



# ----------------------------------------------------------------------
# a function for creating the grids based on the paper
# ----------------------------------------------------------------------
def grid_paper(coords, name):
    '''
    This function takes a list of coordinates, and
    a name, and saves the face grid within a 25X25
    white frame. 
    '''
    # set the canvas
    canvas = np.ones((25,25))*255

    # grid coordinates
    x = coords[0]
    y = coords[1]
    w = coords[2]
    h = coords[3]
    
    # mask the face location
    canvas[y:y+h, x:x+w] = 0

    # set the path to save the grids
    path = 'Data_Mini_4Os/grid_paper/' + name + '.jpg'
    # save the face grid
    plt.imsave(path, canvas, cmap='gray');
    


# ----------------------------------------------------------------------
# a function for converting color pathes to grayscale
# ----------------------------------------------------------------------
def gray(path_read, path_write):
    '''
    The function takes a path to a folder
    (path_read: string), read images in the folder,
    convert them to grayscale versions, and save
    them in another folder (path_write: string)
    '''
    # make a list of the images in the folder
    images = [name for name in os.listdir(path_read)]
    
    # for each image
    for image in images:
        # set the path to the color image
        read = path_read + image
        # read the image
        image_color = cv2.imread(read)
        # generate the grayscale version of the image
        image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
        # set the path for writing grayscale image
        write = path_write + image
        # and save the grayscale image
        cv2.imwrite(write, image_gray)
    


# ----------------------------------------------------------------------
# a function for reading images and saving them as pickle files
# ----------------------------------------------------------------------
def pickle_write(path, name):
    '''
    The function takes a path to a folder
    (string), read images in that folder,
    turns them all to one np.array, and saves
    it as a pickle file under the give 'name'.
    '''
    # create an empty list for adding images' data
    data = []
    
    # for each frame in the list
    for frame in tqdm(frames_list):
        # read the image
        img = plt.imread(path + frame)
        # add it to the data list
        data.append(img)
    
    # convert the list to an array
    data = np.array(data)
    # normalize the data
    data = data / 255
    # change the type from float64 (default)
    # to float32 to lighten the computations loads
    data = data.astype('float32')

    # write the result array as a pickle file
    with open('./Data_Mini_4Os/pickles/' + name + '.pkl','wb') as file:
        pickle.dump(data, file)
    

****

# 1. Generate the Dataset

## 1.1. Paper Scenario

### Pick/Save Random Frames

In [37]:
# make a list of the folders in the data directory
folders = [name for name in os.listdir('Data_MIT/')]
# drop the irrelevant file in the data directory
folders.remove('LICENSE.md')
folders.remove('README.md')

# create an empty list to collect randomly picked
# frames from four orientations
frames = []
# for each orientation
for orientation in range(1,5):    
    # create a list of valid frames with desired orientation in folders
    frames_all = []
    for folder in tqdm(folders):
        # make a list of valid frames
        indices = idx_finder(folder, orientation)
        frames_folder = [folder + '-' + (5 - len(str(idx))) * '0' + str(idx) for idx in indices]
        frames_all = frames_all + frames_folder

    # pick the 10,000 of the frames, randomly
    frames_oriented = random.sample(frames_all, 2500)
    frames = frames + frames_oriented 

frames.sort()

# create a dictionary of folders (keys) and frames (values-list)
# first create a list of the folders
folders = sorted(list(set([frame[:5] for frame in frames])))
# the create an empty dictionary for saving the frames data
frames_dict = {}
# iterate over the folders and store each folder's picked frames
for folder in folders:
    frames_dict[folder] = [frame[-5:] for frame in frames if frame[:5]==folder]
    
# save the dictionary
with open('Data_Mini_4Os/frames_dict.pkl','wb') as f:
    pickle.dump(frames_dict, f)
    
# create a flat list of the frames: folder-frame.jpg in string
frames_list = [frame + '.jpg' for frame in frames]

# and save that list as well
with open('Data_Mini_4Os/frames_list.pkl','wb') as f:
    pickle.dump(frames_list, f)

# copy-paste sampled frames to the dataset folder
for key, val in tqdm(frames_dict.items()):
    for frame in val:
        copy = './Data_MIT/' + key + '/frames/' + frame + '.jpg'
        paste = './Data_Mini_4Os/frames/' + key + '-' + frame + '.jpg'
        shutil.copyfile(copy, paste)

### Extract/Save the Patches

In [52]:
# list of the json files to read
json_list = ['Face', 'LeftEye', 'RightEye']

# iterate over folders in dict and frames in folders
for key, value in tqdm(frames_dict.items()):
    
    for idx in value:
        
        # define the dictionary for recording the info
        coords = {}
        
        # iterate over all json files and read/record the info
        for jsn in json_list:
            # read each json file
            json_reader(coords, jsn, key + '-' + idx)

        # adjust the eyes' coordinates to express the location
        # for within the frame
        coords['LeftEye'][0] += coords['Face'][0]
        coords['LeftEye'][1] += coords['Face'][1]
        coords['RightEye'][0] += coords['Face'][0]
        coords['RightEye'][1] += coords['Face'][1]

        # set the path to the image
        path = 'Data_Mini_4Os/frames/' + key + '-' + idx + '.jpg'
        # read the image
        img = cv2.imread(path)
        
        # iterate over two eyes
        for patch in json_list:
            # set the save path
            save_path = 'Data_Mini_4Os/color_patches/' + patch + '/' + key + '-' + idx + '.jpg' 
            # save the patch
            patch_reader(img, coords[patch], save_path)


### Extract/Save the Paper Grids

In [55]:
# iterate over folders in dict and frames in folders
for key, value in tqdm(frames_dict.items()):
    for idx in value:
        # get the grid coordinates
        coords = get_coords(key, idx)
        # save the grid
        grid_paper(coords, key + '-' + idx )


## 1.2. Grayscale Scenario

In [58]:
# set the root path to the color patches' folder
path_read = './Data_Mini_4Os/color_patches/'
# set the root path to the grayscale patches' folder 
path_write = './Data_Mini_4Os/gray_patches/'

# create a list of patch folders' names
patches = ['face', 'left_eye', 'right_eye']

for patch in patches:
    gray(path_read+patch+'/',
         path_write+patch+'/')

## 1.3. MTCNN Scenario

### Get/Save MTCNN Data for the Frames

In [ ]:
# set the path to the folder
path = './Data_Mini_4Os/frames/'
# get a list of frames in the folder
frames = [name for name in os.listdir(path)]

# create an empty list to save the data
mtcnn_data = []

# for each frame 
for frame in tqdm(frames):
    # read the image
    img = plt.imread(path + frame)
    # get the MTCNN data
    results = MTCNN().detect_faces(img)
    # add the frame name to the MTCNN data list
    results.append(frame)
    
    # add the frame's MTCNN data to the final data list 
    mtcnn_data.append(results)

# sort the list regarding the frames' names
mtcnn_data.sort(key=itemgetter(-1))

# and save the final data list as a pickle file
with open('./Data_Mini_4Os/mtcnn_data.pkl','wb') as file:
    pickle.dump(mtcnn_data, file)


### Check the MTCNN data for Unsuccessful Detections

In [77]:
# create a list to record the frames with failed mtcnn detection
mtcnn_failures = []

# for each list in MTCNN data
for item in mtcnn_data:
    # check the length of the list
    # if it's one
    if len(item) == 1:
        # no face was detected,
        # so add the frame name to the failures list
        mtcnn_failures.append(item[0])
        # drop the item
        mtcnn_data.remove(item)
        # and also print the failed case
        print(item)

        
# update the frames dictionary and list to remove
# the MTCNN failed frames and over-write the pickle files
# ----------------------------------------------------------------------
# for each frame in the failures list
for frame in mtcnn_failures:
    
    # drop the frame (element in value list) from
    # the folder (key) in the frames dictionary 
    frames_dict[frame[:5]].remove(frame[-9:-4])

    # and also drop the frame from the frames_list
    frames_list.remove(frame)
    
# save the frames dictionary
with open('Data_Mini_4Os/frames_dict.pkl','wb') as file:
    pickle.dump(frames_dict, file)
# save the frames list
with open('Data_Mini_4Os/frames_list.pkl','wb') as file:
    pickle.dump(frames_list, file)
# save the MTCNN data
with open('Data_Mini_4Os/mtcnn_data.pkl','wb') as file:
    pickle.dump(mtcnn_data, file)

['01366-00923.jpg']
['02155-00966.jpg']
['02155-01108.jpg']
['02364-00414.jpg']
['02413-01452.jpg']
['02467-00196.jpg']
['02522-01390.jpg']
['02522-01436.jpg']
['02522-01443.jpg']


In [304]:
# so, later on I found another problem due to a
# mis-detection in the original dataset (one that
# should have been tagged as invalid, but it was tagged
# valid and while the right eye is totally missing, it
# gave another set of coordinate on left eye for the
# right eye. Now I'm gonna manually remove that frame
# here and re-write the pickel files
# -----------------------------------------------------
faulty_frame = '02967-02728.jpg'

# drop the frame (element in value list) from
# the folder (key) in the frames dictionary 
frames_dict[faulty_frame[:5]].remove(faulty_frame[-9:-4])

# drop the frame from the frames_list
frames_list.remove(faulty_frame)

# and also drop the item from the MTCNN data
# check all the items in the list
for item in mtcnn_data:
    # if the frame name of the item
    # is equal to the faulty data
    if item[-1] == faulty_frame:
        # drop the item
        mtcnn_data.remove(item)

# save the frames dictionary
with open('Data_Mini_4Os/frames_dict.pkl','wb') as file:
    pickle.dump(frames_dict, file)
# save the frames list
with open('Data_Mini_4Os/frames_list.pkl','wb') as file:
    pickle.dump(frames_list, file)
# save the MTCNN data
with open('Data_Mini_4Os/mtcnn_data.pkl','wb') as file:
    pickle.dump(mtcnn_data, file)

### Create/Save MTCNN Grids

In [246]:
# for each detection in MTCNN data
for item in tqdm(mtcnn_data):
    
    # set the plot
    ax = plt.gca()
    # read the corresponding frame's screen.json file
    with open('./Data_MIT/' + item[-1][:5] + '/screen.json', 'r') as file:
        screen = json.load(file)
    # to check the orientation to get the frame dimensions
    orientation = screen['Orientation'][int(item[-1][-9:-4])]

    # if the orientation is 1 or 2
    if orientation in [1,2]:
        # this is a portrait frame, and we
        # create a portrait canvas of the same size
        canvas = np.ones((640,480))*255
    # otherwise
    else:
        # a landscape frame, and we create
        # a landscape canvas of the same size
        canvas = np.ones((480,640))*255
    
    # it may be the case that more than one face is detected
    # in the frame, in that case we check the size of face boxes
    # and only keep the largest box as the subject face
    # -------------------------------------------------
    # keep doing until there's only one face detection
    # (along with the frame name) left in the item
    while len(item) != 2:
        # check the first two faces' width
        if item[0]['box'][2] > item[1]['box'][2]:
            # drop the second one if the first one is bigger
            item.pop(1)
        # or
        else:
            # drop the first one if the second one is bigger
            item.pop(0)

            
    # get the facial landmark coordinates
    # --------------------------------------------
    # get the facebox top-left corner x coordinate
    x = item[0]['box'][0]
    # get the facebox top-left corner y coordinate
    y = item[0]['box'][1]
    # get the face box width
    width = item[0]['box'][2]
    # get the face box height
    height = item[0]['box'][3]
    # get the (x,y) coordinates of the left eye
    left_eye = list(item[0]['keypoints']['left_eye'])
    # get the (x,y) coordinates of the right eye
    right_eye = list(item[0]['keypoints']['right_eye'])
    # get the (x,y) coordinates of the nose
    nose = list(item[0]['keypoints']['nose'])

    
    # generate face box rectangle and eyes-nose triangle patches
    # ---------------------------------------------------------------
    # draw the face box as a patch in black
    rect = Rectangle((x, y), width, height, fill=True, color='black')
    # mark the eyes-nose locations as triangle in white
    tri = Polygon([left_eye,right_eye,nose], closed=True, fill=True, color='white')
    
    
    # add the patches to the plot
    ax.add_patch(rect)
    ax.add_patch(tri)
    
    ax.imshow(canvas, cmap='gray_r')
    
    # shut down the axis
    ax.axis('off')
    # and save the image
    plt.savefig('./Data_Mini_4Os/grid_mtcnn/' + item[-1],
                bbox_inches='tight', pad_inches=0, dpi=212)
    
    plt.close();

### Rescale MTCNN Grids

In [256]:
# for each frame in the list
for frame in tqdm(frames_list):
    # set the path to the file
    path = './Data_Mini_4Os/grid_mtcnn/' + frame
    # read the MTCNN original grid
    grid = plt.imread(path)
    # scale it down
    grid = cv2.resize(grid, (50, 50))
    # and over-write it
    #plt.imsave(path, gird)
    plt.imsave(path, grid)


### Create/Save MTCNN Data Point Vector

In [310]:
# create an empty list for recording MTCNN data
mtcnn_vector = []

# for each frame in frames_list
for item in mtcnn_data:
    
    # it may be the case that more than one face is detected
    # in the frame, in that case we check the size of face boxes
    # and only keep the largest box as the subject face
    # -------------------------------------------------
    # keep doing until there's only one face detection
    # (along with the frame name) left in the item
    while len(item) != 2:
        # check the first two faces' width
        if item[0]['box'][2] > item[1]['box'][2]:
            # drop the second one if the first one is bigger
            item.pop(1)
        # or
        else:
            # drop the first one if the second one is bigger
            item.pop(0)

    # create a list to collect MTCNN data for the frame
    data = []
    
    # get the (x,y) coordinates of the left eye
    left_eye = list(item[0]['keypoints']['left_eye'])
    # add the left eye data to the list
    data = data + left_eye
    # get the (x,y) coordinates of the right eye
    right_eye = list(item[0]['keypoints']['right_eye'])
    # add the right eye data to the list
    data = data + right_eye
    # get the (x,y) coordinates of the nose
    nose = list(item[0]['keypoints']['nose'])
    # add the nose data to the list
    data = data + nose
    
    # add frame data list to the final MTCNN data list
    mtcnn_vector.append(data)


# convert the final list to an np.array
mtcnn_vector = np.array(mtcnn_vector)

# and save the vector as a pickle file
with open('Data_Mini_4Os/pickles/mtcnn_vector.pkl','wb') as file:
    pickle.dump(mtcnn_vector, file)

## 1.4. Nose Scenario

### Extract the eyes' patches width

In [312]:
# create an empty list for recording frames' eyes'
# patches widths so to cut the nose's patch out 
# with the same size 
eye_patch_widths = []

# for each frame in the list
for frame in tqdm(frames_list):
    # set the path to the json file
    path = './Data_MIT/' + frame[:5]
    # set the index to the frame
    idx = int(frame[-9:-4])
    # read the json file for left eye (both eyes' patches
    # have the same dimensions)
    with open(path + '/appleLeftEye.json', 'r') as file:
        eye_l = json.load(file)
    # get the eye's patch width
    width = eye_l['W'][idx]
    
    # add the width and the frame name to the list
    eye_patch_widths.append([width, frame])


### Extract/Save Nose Patches

In [313]:
# set the path to read the original frames
path_read = './Data_Mini_4Os/frames/'
# set the path to write the nose patches
path_write = './Data_Mini_4Os/color_patches/nose/'

# for 9,991 times:
for idx in tqdm(range(9990)):
    
    # set the path to the frame using the index
    read = path_read + mtcnn_data[idx][-1]
    # read the frame
    img = cv2.imread(read)
    
    # it may be the case that more than one face is detected
    # in the frame, in that case we check the size of face boxes
    # and only keep the largest box as the subject face
    # -------------------------------------------------
    # keep doing until there's only one face detection
    # (along with the frame name) left in the item
    while len(mtcnn_data[idx]) != 2:
        # check the first two faces' width
        if mtcnn_data[idx][0]['box'][2] > mtcnn_data[idx][1]['box'][2]:
            # drop the second one if the first one is bigger
            mtcnn_data[idx].pop(1)
        # or
        else:
            # drop the first one if the second one is bigger
            mtcnn_data[idx].pop(0)
    
    # read the nose coordinates
    x, y = list(mtcnn_data[idx][0]['keypoints']['nose'])
    
    # read the eye patch width and record half its quantity
    half_width = round(eye_patch_widths[idx][0]/2)
    
    # cut out the nose patch
    nose = img[y-half_width:y+half_width,
               x-half_width:x+half_width]
    
    # resize the patch
    nose = cv2.resize(nose, (64, 64))
    
    # set the path to save the nose patch
    write = path_write + mtcnn_data[idx][-1]
    # and finally, save it
    cv2.imwrite(write, nose)
    

****

# 2. Generate the Pickle Files

## 2.1. Inputs

### The Patches

In [321]:
# save the color patches as pickle files
# -----------------------------------------------
# set the path to color patches folder
path_parent = './Data_Mini_4Os/color_patches/'

# make a list of all folders in color_patches folder
folders = ['face', 'left_eye', 'right_eye', 'nose']

# for each folder
for folder in folders:
    # set the path
    path = path_parent + folder + '/'
    # generate the pickle files for the folder
    pickle_write(path, folder)


# save the grayscale patches as pickle files
# -----------------------------------------------
# set the path to grayscale patches folder
path_parent = './Data_Mini_4Os/gray_patches/'

# make a list of all folders in color_patches folder
folders = ['face', 'left_eye', 'right_eye']

# for each folder
for folder in folders:
    # set the path
    path = path_parent + folder + '/'
    # set the name for the pickle file
    name = folder + '_gray'
    # generate the pickle files for the folder
    pickle_write(path, name)


### The Grids

In [324]:
# for each grid in the list
for grid in ['grid_mtcnn', 'grid_paper']:
    
    # set the path to the grids' folder
    path = './Data_Mini_4Os/' + grid + '/'
    
    # create an empty list for adding grids' data
    data = []

    # for each grid in the list
    for frame in tqdm(frames_list):
        # read the grid
        img = plt.imread(path + frame)
        # add it to the data list
        data.append(img)

    # convert the list to an array
    data = np.array(data)
    
    # binarize the grids, cause they're
    # kinda blurry after being saved and read
    # in such small dimensions
    data[data < 128] = 0
    data[data >= 128] = 1
    # remove the unseless color channel
    data = data[:,:,:,0]
    
    # write the result array as a pickle file
    with open('./Data_Mini_4Os/pickles/' + grid + '.pkl','wb') as file:
        pickle.dump(data, file)


## 2.2. Labels

In [326]:
# create a list to store dot/gaze location for each frame
labels = []
    
# iterate over dictionary keys/values and get the labels
for key, value in tqdm(frames_dict.items()):
    
    # set the specific folder's path
    path = 'Data_MIT/' + key
    
    # read the dot json
    with open(path + '/dotInfo.json', 'r') as file:
        dot = json.load(file)
    
    # convert the frames list from string to integer
    indices_int = [int(idx) for idx in value] 
    
    # for each index in the list
    for idx in indices_int:
        # add the X and Y values to the list
        labels.append([dot['XCam'][idx], dot['YCam'][idx]])

        
# convert the list to an np.array
labels = np.array(labels)

# and save it as a pickle file
with open('Data_Mini_4Os/labels.pkl','wb') as f:
    pickle.dump(labels, f)
    